In [26]:
import os
from Pylice_def.data import page as data_page
import Pylice_def.data.preprocess as prep
from Pylice_def.forecast import modelling
from Pylice_def.forecast import forecasts
import pandas as pd 
import numpy as np 
from Pylice_def.queing import simulation as sim
from Pylice_def.queing import page as q_page

from Pylice_def.shift_scheduling import preprocess as sched_prep
from Pylice_def.shift_scheduling.optimization import pylice_opt_model_soft, optimize_soft
from Pylice_def.shift_scheduling import page as shift_page
import pyomo.environ as pyo
import matplotlib.pyplot as plt
import datetime as dt
import random as rd
from tqdm import tqdm
from copy import copy

def optimization_preprocess(n_weeks, earliest_shift,latest_shift, allowed_lens, minimum_night, minimum_day, demand, f):
    allowed_lens = set(list(map(int, allowed_lens)))
    ts = sched_prep.horizon_timestamps(n_weeks,f)
    possible_shifts, starts_ends = sched_prep.create_possible_shifs(ts, sched_prep.create_forbidden_shifts(earliest_shift,latest_shift, f),allowed_lens, f)
    shifts_info = sched_prep.compute_shift_statistics(possible_shifts, f)
    shifts_info["Start_end"] = starts_ends
    possible_shifts = sched_prep.change_column_names(possible_shifts,ts,f)
    min_demand = sched_prep.create_min_demand(ts, minimum_night, minimum_day, f)
    shift_costs = shifts_info["cost"].copy(deep = True)
    shift_costs = shift_costs.to_dict()
    S = possible_shifts.index.tolist()
    ts = possible_shifts.columns.tolist()
    demand.index = ts
    min_demand.index = ts
    demand = pd.concat([demand, min_demand]).groupby(level=0).max().sort_index().to_dict()["Staffing_level"]
    shift_day_match_start, shift_day_match_end = sched_prep.match_starting_ending(ts, shifts_info)
    day_shift_match = sched_prep.create_coverage(possible_shifts)
    return ts, S, demand, shift_costs,day_shift_match, shift_day_match_start, shift_day_match_end, possible_shifts, shifts_info

def tabularize_results(model, shift_stats, ts_horizon, overlap, shift_lenghts, f):
    choosen = []
    for i in model.ShiftSelect:
        choosen.append((i,pyo.value(model.ShiftSelect[i]) , pyo.value(model.Allocated_Teams[i])))
    choosen = pd.DataFrame(choosen).rename(columns = {1 : "selected", 2 : "staff"})
    # print(choosen)
    choosen = choosen.loc[(choosen["selected"] > 0) & (choosen["staff"] > 0)].set_index(0, drop = True).drop(columns= "selected", axis = 1)
    choosen["start_num"] = list(map(lambda shift: shift_stats.loc[shift, "Start_end"][0], choosen.index.tolist()))
    new_end = list(map(lambda shift: shift_stats.loc[shift, "Start_end"][1] +1 , choosen.index.tolist()))
    for n, i in enumerate(new_end):
        if i == len(ts_horizon):
            new_end[n] = 0
    choosen["end_num"] = new_end
    choosen["start"] = choosen["start_num"].apply(lambda x: shift_page.from_num_to_date(x, f))
    choosen["end"] = choosen["end_num"].apply(lambda x :shift_page.from_num_to_date(x, f))
    choosen["len"] = (choosen["end"] - choosen["start"]).dt.seconds/(60*f)
    choosen = choosen[["staff", "start_num", "end_num", "len", "start", "end"]]
    choosen.sort_values("start_num", inplace = True)
    target = dt.timedelta(minutes = overlap)
    max_allowed_len = max(shift_lenghts)
    ##
    # shifts = shift_page.create_overlap(choosen, target, max_allowed_len)
    shifts = list(map(list, choosen.reset_index().to_numpy()))
    shifts = pd.DataFrame(shifts)
    ##
    shifts.columns = choosen.reset_index(names= ["Shift_name"]).columns
    shifts.drop("len", axis = 1, inplace = True)
    shifts.set_index("Shift_name", inplace = True)
    shifts["start-end"] = shifts["start"].dt.time.astype(str) +  " - " + shifts["end"].dt.time.astype(str)

    layover = shifts[shifts["start_num"] > shifts["end_num"]].index.tolist()
    for ind in layover:
        shifts.loc[f"{ind}-->"] =   shifts.loc[ind, "staff"],\
                                    shifts.loc[ind, "start_num"],\
                                    0,\
                                    shifts.loc[ind, "start"],\
                                    pd.to_datetime("2023-01-01 00:00:00"),\
                                    shifts.loc[ind, "start-end"]
        
        shifts.loc[f"-->{ind}"] =   shifts.loc[ind, "staff"],\
                                    0,\
                                    shifts.loc[ind, "end_num"],\
                                    pd.to_datetime("2023-01-01 00:00:00"),\
                                    shifts.loc[ind, "end"],\
                                    shifts.loc[ind, "start-end"]
        shifts.drop(ind, inplace = True)

    shifts["len"] = (shifts["end"] - shifts["start"]).dt.seconds/(60*15)
    coverage = shifts[["staff", "start", "end"]].copy()
    ts_ = list(pd.date_range(start = "2023-01-01 00:00:00", end= "2023-01-08 00:00:00", freq = "15min"))
    shifts["start_num"] = shifts["start"].apply(lambda x: (ts_.index(x)))
    shifts.drop(["start", "end"], axis = 1, inplace = True)
    layover = shifts.loc[shifts["len"] < 0].index.tolist()
    for i in layover:
        shifts.loc[i, "len"] = len(ts_) - shifts.loc[i, "start_num"] ###
    shifts['color'] = shifts.apply(lambda df: shift_page.color(df), axis=1)
    shifts.reset_index(names = ["Shift_name"], drop = False, inplace= True)
    shifts.sort_values("start-end", inplace = True, ascending= False)
    shifts["end_num"] = shifts["start_num"] + shifts["len"] 
    return shifts, coverage

def tabularize_results_heu(model, shift_stats, ts_horizon, overlap, shift_lenghts, f):
    choosen = []
    for i in model:
        choosen.append((i[0],int(i[1] > 0) , i[1]))
    choosen = pd.DataFrame(choosen).rename(columns = {1 : "selected", 2 : "staff"})
    choosen = choosen.loc[(choosen["selected"] > 0) & (choosen["staff"] > 0)].set_index(0, drop = True).drop(columns= "selected", axis = 1)
    choosen["start_num"] = list(map(lambda shift: shift_stats.loc[shift, "Start_end"][0], choosen.index.tolist()))
    new_end = list(map(lambda shift: shift_stats.loc[shift, "Start_end"][-1] +1 , choosen.index.tolist()))
    for n, i in enumerate(new_end):
        if i == len(ts_horizon):
            new_end[n] = 0
    choosen["end_num"] = new_end
    choosen["start"] = choosen["start_num"].apply(lambda x: shift_page.from_num_to_date(x, f))
    choosen["end"] = choosen["end_num"].apply(lambda x :shift_page.from_num_to_date(x, f))
    choosen["len"] = (choosen["end"] - choosen["start"]).dt.seconds/(60*f)
    choosen = choosen[["staff", "start_num", "end_num", "len", "start", "end"]]
    choosen.sort_values("start_num", inplace = True)
    target = dt.timedelta(minutes = overlap)
    max_allowed_len = max(shift_lenghts)
    ##
    # shifts = shift_page.create_overlap(choosen, target, max_allowed_len)
    shifts = list(map(list, choosen.reset_index().to_numpy()))
    shifts = pd.DataFrame(shifts)
    ##
    shifts.columns = choosen.reset_index(names= ["Shift_name"]).columns
    shifts.drop("len", axis = 1, inplace = True)
    shifts.set_index("Shift_name", inplace = True)
    shifts["start-end"] = shifts["start"].dt.time.astype(str) +  " - " + shifts["end"].dt.time.astype(str)

    layover = shifts[shifts["start_num"] > shifts["end_num"]].index.tolist()
    for ind in layover:
        shifts.loc[f"{ind}-->"] =   shifts.loc[ind, "staff"],\
                                    shifts.loc[ind, "start_num"],\
                                    0,\
                                    shifts.loc[ind, "start"],\
                                    pd.to_datetime("2023-01-01 00:00:00"),\
                                    shifts.loc[ind, "start-end"]
        
        shifts.loc[f"-->{ind}"] =   shifts.loc[ind, "staff"],\
                                    0,\
                                    shifts.loc[ind, "end_num"],\
                                    pd.to_datetime("2023-01-01 00:00:00"),\
                                    shifts.loc[ind, "end"],\
                                    shifts.loc[ind, "start-end"]
        shifts.drop(ind, inplace = True)

    shifts["len"] = (shifts["end"] - shifts["start"]).dt.seconds/(60*15)
    coverage = shifts[["staff", "start", "end"]].copy()
    ts_ = list(pd.date_range(start = "2023-01-01 00:00:00", end= "2023-01-08 00:00:00", freq = "15min"))
    shifts["start_num"] = shifts["start"].apply(lambda x: (ts_.index(x)))
    shifts.drop(["start", "end"], axis = 1, inplace = True)
    layover = shifts.loc[shifts["len"] < 0].index.tolist()
    for i in layover:
        shifts.loc[i, "len"] = len(ts_) - shifts.loc[i, "start_num"] ###
    shifts['color'] = shifts.apply(lambda df: shift_page.color(df), axis=1)
    shifts.reset_index(names = ["Shift_name"], drop = False, inplace= True)
    shifts.sort_values("start-end", inplace = True, ascending= False)
    print(f"Maximum shift length is respected: {(shifts.len > 4* max_allowed_len).unique().tolist() == [False]}")
    shifts["end_num"] = shifts["start_num"] + shifts["len"] 
    return shifts, coverage

def plot_staffing_requirements(demand, params,t):
        q = 60//t
        demand["how"] = demand.reset_index(inplace = False).index
        axs = plt.subplot()
        y_max = demand.Staffing_level.max() +2 
        y_min = max(0, demand.Staffing_level.min() -2) 
        plt.plot(demand["how"], demand["Staffing_level"],zorder = 2, color = "green")
        for day in range(0,(24*7*q), 24*q):
                axs.vlines(x = day, ymin = 0, ymax =y_max + 0.2, color = "blue", zorder = 1, alpha = .4, linestyles= "dashed")
        # axs.hlines(y = 3, xmin=demand["how"].min(), xmax = demand["how"].max(), color = "tomato", zorder = 2)
        axs.set_xticks(list(range(1,(24*7*q)+1, 24*q)))
        xticks_minor = list(range(0 ,(24*7*q)+1,q))
        axs.set_xticks(xticks_minor, minor=True)
        axs.set_xticklabels(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
        axs.set_xlim(xmin= 0, xmax = 24*7*q)
        axs.set_ylim(ymin= 1, ymax = y_max -1)
        axs.set_facecolor("lightgray")
        axs.set_ylabel("Number of employees")
        axs.set_xlabel("Average Week")
        plt.show()
        print("Call Type:", call_type)
        for p in params.keys():
                text = f"{p.title()}: {params[p][call_type]}"
                if p in {"asa", "aht"}:
                        text += " seconds"
                elif p in {"max_occupancy", "shrinkage", "service_level"}:
                        text += "%"
                print(text)

def create_range(cell, times):
    start, end = cell[0], cell[1] + 1
    if start < end:
        return list(range(start, end))
    else: 
        return list(range(start, len(times))) + list(range(0, end))

def create_demand_fv(solution, demand, t_time, shift_stats):  
    tmp = shift_stats.reset_index(names = "Shift_name")[["Shift_name", "Start_end"]].copy()
    tmp = pd.merge(pd.DataFrame(solution[:-1]).rename(columns = {0:"Shift_name",1:"Assigned", "string" : "Shift_name", "integer" : "Assigned"}), tmp, on = "Shift_name", how = "left")
    tmp = tmp.explode("Start_end")
    tmp["Start_end"] = tmp.Start_end.apply(lambda y: t_time[y])
    tmp["Demand"] = tmp.Start_end.apply(lambda y: demand[y])
    tmp = tmp.groupby("Start_end").agg({"Demand" : "mean", "Assigned" : "sum"})
    tmp["UnderStaff"] = tmp["Demand"] - tmp["Assigned"]
    return tmp.UnderStaff.abs().sum()

def get_objFunc(solution, cost_df,demand, demand_weigth, t_time, shift_stats):
    cost_weight = 1 - demand_weigth
    demand_diff_weight = demand_weigth
    minimum_cost = 213727.5 # Max_cap = 15
    maximum_cost = 1070317.5 # Max_cap = 15
    minimum_demand_diff = 361 # Max_cap = 15
    maximum_demand_diff = 3459 # Max_cap = 15, 3459
    cost_normalization_range = maximum_cost - minimum_cost
    demand_diff_normalization_range = maximum_demand_diff - minimum_demand_diff
    cost =  sum(list(map(lambda x : cost_df[x[0]] * x[1], solution)))
    demand_diff = create_demand_fv(solution, demand, t_time, shift_stats)
    return 1000 * (cost_weight*((cost - minimum_cost)/cost_normalization_range) + demand_diff_weight*((demand_diff - minimum_demand_diff)/demand_diff_normalization_range))

def neighbour_move(solution, moves_used):
    for move in moves_used:
        if solution[:-1].tolist() == moves_used[move].tolist():
            try:
                return (move[1], move[0])
            except:
                return (move[0][1], move[0][0])
    raise BaseException("STOOOOOP")

def get_final_objs(solution, cost_df, demand, shift_stats, time_t):
    cost =  sum(list(map(lambda x : cost_df[x[0]] * x[1], solution)))
    tmp = shift_stats.reset_index(names = "Shift_name")[["Shift_name", "Start_end"]].copy()
    tmp = pd.merge(pd.DataFrame.from_records(solution).rename(columns = {0:"Shift_name",1:"Assigned", "string" : "Shift_name", "integer" : "Assigned"}), tmp, on = "Shift_name", how = "left")
    tmp = tmp.explode("Start_end")
    tmp["Start_end"] = tmp.Start_end.apply(lambda y: time_t[y])
    tmp["Demand"] = tmp.Start_end.apply(lambda y: demand[y])
    tmp = tmp.groupby("Start_end").agg({"Demand" : "mean", "Assigned" : "sum"})
    tmp["UnderStaff"] = tmp["Demand"] - tmp["Assigned"]
    demand_diff =  tmp.UnderStaff.abs().sum()
    return cost, demand_diff/len(demand)

# Soft Model Run
def run_soft(earliest, latest, shift_lenghts, minimum_night, minimum_day, demand, timeframe, teamsize, max_cap, overlap, demand_weigth):
    shift_lenghts = list(map(lambda x: x*(60//timeframe), shift_lenghts))
    ts_horizon, S,\
        demand, shift_costs,\
            t_s_cov, starting_shfits,\
                ending_shifts, possible_shifts,\
                    shift_stats = optimization_preprocess(n_weeks = 1,
                                                        earliest_shift = earliest,
                                                        latest_shift = latest,
                                                        allowed_lens = shift_lenghts,
                                                        minimum_night= minimum_night,
                                                        minimum_day = minimum_day,
                                                        demand = demand.copy(),
                                                        f = timeframe)

    opt_model = pylice_opt_model_soft(time_t = ts_horizon,
                                    p_shifts = S,
                                    demand = demand,
                                    teamsize = teamsize,
                                    cap_max = max_cap,
                                    shift_cost = shift_costs,
                                    day_shift_match= t_s_cov,
                                    shift_day_match_start= starting_shfits,
                                    shift_day_match_end = ending_shifts,
                                    demand_weigth = demand_weigth)
    # print("Optimization- begins")
    opt_model, status = optimize_soft(opt_model, optimize = "xdxd")
    results, n_assigned = tabularize_results(opt_model, shift_stats, ts_horizon, overlap, shift_lenghts, timeframe)

    ts_ = list(forecasts.create_horizon_dates("2023-01-01", 1, 15))
    n_assigned = results[["start_num", "end_num", "staff"]].astype(int).copy()
    n_assigned["coverage"] = n_assigned[["start_num", "end_num"]].apply(lambda df: ts_[df["start_num"] : df["end_num"]], axis = 1)
    n_assigned =  n_assigned.explode("coverage")
    n_assigned = n_assigned.groupby("coverage").agg({"staff":"sum"}).sort_index()
    n_assigned.rename({"staff": "Assigned"}, axis = 1, inplace = True)
    n_assigned = n_assigned.sort_index().resample("15min").ffill()

    demand = pd.DataFrame.from_dict(demand.items()).rename(columns = {0: "Date_time", 1 : "Staffing_level"})
    demand.set_index("Date_time", inplace = True)
    mapping = {"Monday" : "1", "Tuesday" : "2", "Wednesday" : "3", "Thursday" : "4", "Friday" : "5", "Saturday" : "6", "Sunday" : "7"}
    demand.index = list(map(lambda x : pd.to_datetime("2023-01-0" + mapping[x.split(" ")[0]] + " " + x.split(" ")[1]), demand.index.tolist()))
    demand = demand.sort_index().resample("15min").ffill()

    n_assigned = pd.merge(n_assigned, demand, right_index = True, left_index = True, how = "outer").ffill()
    n_assigned.reset_index(drop = False, names = ["date"], inplace = True)

    # schedule_graph = shift_page.plot_resulting_schedule(results.sort_values("start-end"), n_assigned)
    # plt.show()
    n_assigned["OverStaffing"] = n_assigned["Assigned"] - n_assigned["Staffing_level"]
    # print(f"Average overstaffing: {n_assigned.OverStaffing.mean()}")
    # print(f"Maximum overcoverage: {n_assigned.OverStaffing.max()}")
    # print(f"Total cost: {pyo.value(opt_model.Tot_Costs)}")
    return n_assigned, pyo.value(opt_model.Tot_Costs)

# Tabu Search Run
def build_initial_solution(earliest, shift_day_match_start, t_time, shift_stats, demand,shift_day_match_end, teamsize, meta_dict, S):
    build_initial = []
    end = None
    start = f"Monday {str(earliest).zfill(2)}:00:00"
    while start in list(shift_day_match_start.keys()):
        pox = list(shift_day_match_start[start])
        shift = rd.sample(pox, 1)[0]
        end = t_time[shift_stats.loc[shift, "Start_end"][-1]]
        while end not in list(shift_day_match_start.keys()):
            pox.remove(shift)
            shift = rd.sample(pox, 1)[0]
            end = t_time[shift_stats.loc[shift, "Start_end"][-1]]
        start = end
        build_initial.append(shift)
        if "Sunday 20:30:00" <= start < "Sunday 23:00:00":
            break
    a = set(shift_day_match_start[t_time[shift_stats.loc[build_initial[-1], "Start_end"][-1]]]).intersection(set(shift_day_match_end[f"Monday {str(earliest).zfill(2)}:30:00"]))
    build_initial.append(list(a)[0])
    build_initial = pd.DataFrame(build_initial).rename(columns = {0: "Shift_name"})
    build_initial["coverage"] = build_initial["Shift_name"].apply(lambda x: meta_dict[x])
    build_initial = build_initial.explode("coverage").reset_index(drop = True)
    build_initial["coverage"] = build_initial["coverage"].apply(lambda x : t_time[x])
    build_initial = pd.merge(build_initial, demand, left_on = "coverage", right_index = True, how = "left")
    build_initial["Assigned"] = teamsize * (max(build_initial.Staffing_level.mean() // teamsize,1))
    build_initial_demand = build_initial.groupby("coverage").agg({"Assigned":"sum", "Staffing_level":"mean", "Assigned":"sum", "Shift_name": "unique"})
    build_initial_demand["order"] = list(map(lambda x : t_time.index(x), build_initial_demand.index))
    additional_pp = set(map(lambda x: x[0], build_initial_demand.loc[(build_initial_demand.Staffing_level > build_initial_demand.Staffing_level.mean()) & (build_initial_demand.Assigned < build_initial_demand.Staffing_level)].sort_values("order").Shift_name))
    build_initial.loc[build_initial.Shift_name.isin(additional_pp), "Assigned"] = teamsize * (max(build_initial.Staffing_level.mean() // teamsize,1)) *2
    assert len(build_initial[["Shift_name", "Assigned"]].drop_duplicates()["Shift_name"].value_counts().unique()) == 1
    Initial_Solution = []
    for shift, assigned in build_initial[["Shift_name", "Assigned"]].drop_duplicates().to_numpy().tolist():
        Initial_Solution.append((shift, assigned))

    for unused in set(S).difference(set(map(lambda y: y[0], Initial_Solution))):
        Initial_Solution.append((unused, 0))
        
    return Initial_Solution

def get_allocated_for_shift_in_solution(solution, shift):
    return solution[list(map(lambda x: x[0], solution)).index(shift)][1]

def ReplaceShifts(solution0, all_shifts, N = 20):
    shifts_subs = []
    potentialss = []
    for i in solution0:
        if i[1] > 0:
            shifts_subs.append(i[0])
        else:
            potentialss.append(i[0])
    List_of_N = []    
    potentials = set(potentialss)
    for shift in shifts_subs:
        allocated = get_allocated_for_shift_in_solution(solution0, shift)
        replacement = (rd.sample(list(potentials), 1)[0],allocated )
        potentials.remove(replacement[0])
        while (replacement[0], 0) not in solution0.tolist():
            replacement = (rd.sample(list(potentials), 1)[0], allocated)
        List_of_N.append(((shift, allocated), replacement))
    moves_used = {}
    neighbourhood = []
    for i in range(N):
        if len(List_of_N) <= 1:
            continue
        solution = copy(solution0)
        a = rd.randint(min(len(List_of_N), 2),min(6,len(List_of_N)))
        Chosen_Changes = rd.sample(List_of_N, a) # diversification
        for old, new in Chosen_Changes:
            solution[solution.tolist().index(old)] = (solution[solution.tolist().index(old)][0], 0)
            solution[solution.tolist().index((new[0], 0))] = (solution[solution.tolist().index((new[0], 0))][0], new[1])
        neighbourhood.append(solution)
        moves_used[tuple(Chosen_Changes)] = np.array(solution, dtype = [('string', 'U20'), ('integer', int)])
    return np.array(neighbourhood, dtype = [('string', 'U20'), ('integer', int)]), moves_used

def ChangeTeamSize(solution0, teamsize, demand, N = 20):
    sizes = [teamsize * n for n in range(1,int((max(demand.values())//teamsize )+1))]
    shifts_subs = []
    for i in solution0:
        if i[1] > 0:
            shifts_subs.append(i[0])
    moves_used = {}
    neighbourhood = []
    choosen_shifts = rd.sample(shifts_subs, len(shifts_subs)//2)
    for i in range(N):
        solution = copy(solution0)
        Chosen_Changes = []
        for shift in choosen_shifts:
            new = (shift, rd.sample(sizes, 1)[0])
            old = (shift, get_allocated_for_shift_in_solution(solution, shift))
            solution[solution.tolist().index(old)] = new
            Chosen_Changes.append((old, new))
        neighbourhood.append(solution)
        moves_used[tuple(Chosen_Changes)] = np.array(solution, dtype = [('string', 'U20'), ('integer', int)])
    return np.array(neighbourhood, dtype = [('string', 'U20'), ('integer', int)]), moves_used

def Number_of_Shifts(solution0, teamsize, all_shifts,meta_dict, N = 20):
    active_shifts_subs = []
    for i in solution0:
        if i[1] > 0:
            active_shifts_subs.append(i[0])

    inactive_shifts_subs = []
    for i in solution0:
        if i[1] == 0:
            inactive_shifts_subs.append(i[0])
    neighbourhood = []
    moves_used = {}
    for i in range(N):
        active_cov = {1}
        inactive_cov = {2}
        solution = copy(solution0)
        Chosen_Changes = []
        while not active_cov.issubset(inactive_cov):
            choosen_to_drop = rd.sample(active_shifts_subs, rd.randint(min(len(active_shifts_subs), 2),min(6,len(active_shifts_subs) )))
            potentials = []
            for shift in choosen_to_drop:
                potentials.extend(inactive_shifts_subs)
            potentials = set(potentials)

            choosen_to_add = rd.sample(inactive_shifts_subs, rd.randint(min(len(inactive_shifts_subs), 2),min(6,len(inactive_shifts_subs) )))
            while len(choosen_to_add) == len(choosen_to_drop):
                choosen_to_add = rd.sample(inactive_shifts_subs, rd.randint(min(len(inactive_shifts_subs), 2),min(6,len(inactive_shifts_subs) )))

            active_cov = set().union(*[set(meta_dict[p]) for p in choosen_to_drop])
            inactive_cov = set().union(*[set(meta_dict[p]) for p in choosen_to_add])
        
        for shift in choosen_to_add:
            new = (shift, teamsize)
            old = (shift, get_allocated_for_shift_in_solution(solution, shift))
            solution[solution.tolist().index(old)] = new
            Chosen_Changes.append((old,new))
        for shift in choosen_to_drop:
            new = (shift, 0)
            old = (shift, get_allocated_for_shift_in_solution(solution, shift))
            solution[solution.tolist().index(old)] = new
            Chosen_Changes.append((old,new))
        neighbourhood.append(solution)
        moves_used[tuple(Chosen_Changes)] = np.array(solution, dtype = [('string', 'U20'), ('integer', int)])
    return np.array(neighbourhood, dtype = [('string', 'U20'), ('integer', int)]), moves_used

def neighbour_move(solution, moves_used):
    for move in moves_used:
        if solution[:-1].tolist() == moves_used[move].tolist():
            try:
                return (move[1], move[0])
            except:
                return (move[0][1], move[0][0])
    raise BaseException("STOOOOOP")

def EnsabledMove(solution0, teamsize, demand, all_shifts ,meta_dict, N = 20):
    All_N_for_i, old_moves = Number_of_Shifts(solution0, teamsize, all_shifts,meta_dict, N)
    new_neighbourhood = []
    moves_used = {}
    for neighbour in All_N_for_i:
        for move in old_moves:
            if neighbour.tolist() == old_moves[move].tolist():
                old_move = move
        new_neighbour, new_move = ChangeTeamSize(neighbour, teamsize, demand, N = 1)
        new_move = list(new_move.keys())[0]
        moves_used[(old_move, new_move)] = np.array(new_neighbour.tolist()[0], dtype = [('string', 'U20'), ('integer', int)])
        new_neighbourhood.append(new_neighbour.tolist()[0])
    return np.array(new_neighbourhood, dtype = [('string', 'U20'), ('integer', int)]), moves_used

def plot_staffing_requirements(demand, params):
        demand["how"] = demand.reset_index(inplace = False).index
        axs = plt.subplot()
        y_max = demand.Staffing_level.max() +2 
        y_min = max(0, demand.Staffing_level.min() -2) 
        plt.plot(demand["how"], demand["Staffing_level"],zorder = 1, color = "green")
        for day in range(0,(24*7*4), 24*4):
                axs.vlines(x = day, ymin = y_min, ymax =y_max + 0.2, color = "blue", zorder = 1, alpha = .4, linestyles= "dashed")
        axs.set_xticks(list(range(12*4,(24*7*4), 24*4)))
        xticks_minor = list(range(0 ,24*7*4,4))
        axs.set_xticks(xticks_minor, minor=True)
        axs.set_xticklabels(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
        axs.set_xlim(xmin= 0, xmax = 24*7*4)
        axs.set_ylim(ymin= y_min, ymax = y_max)
        axs.set_facecolor("lightgray")
        axs.set_ylabel("Number of employees")
        axs.set_xlabel("Average Week")
        plt.show()
        print("Call Type:", call_type)
        for p in params.keys():
                text = f"{p.title()}: {params[p][call_type]}"
                if p in {"asa", "aht"}:
                        text += " seconds"
                elif p in {"max_occupancy", "shrinkage", "service_level"}:
                        text += "%"
                print(text)


def TabuSearch(X0, Runs, ShiftCosts_df, demand, demand_weight, S, t_time, shift_stats, meta_dict):
    X0 = np.array(X0, dtype = [('string', 'U20'), ('integer', int)])
    # Initial_For_Final = X0[:]
    ## Tabu List ## 
    moves_config = {0: "ReplaceShifts(X0, S)",
                    1: "ChangeTeamSize(X0, teamsize = teamsize, demand = demand)",
                    2: "Number_of_Shifts(X0, teamsize = teamsize, all_shifts = S, meta_dict = meta_dict)",
                    3: "EnsabledMove(X0, teamsize = teamsize, demand = demand, all_shifts = S ,meta_dict = meta_dict)"}

    change_move = Runs//len(moves_config.keys()) ##### Change this as part of MDA

    Length_of_Tabu_List = np.random.randint(10,30)

    Iterations = 1
    Tabu_List = []  

    Save_Solutions_Here = []

    for Iteration in range(Runs):
        # print(Iteration)
        All_N_for_i, moves_used = eval(moves_config[int(Iteration//change_move)])
        if len(All_N_for_i) == 0:
            if (Iteration + 1)%5 == 0:
                new_len = np.random.randint(10,30)
                if new_len < Length_of_Tabu_List:
                    Tabu_List = Tabu_List[:new_len]
                Length_of_Tabu_List = new_len
            Iterations += 1
            continue
        ObjFct_Values_for_N = np.array([np.concatenate((s, np.array([("Fitness Value" , get_objFunc(s, ShiftCosts_df, demand, demand_weight, t_time, shift_stats))],  dtype = [('string', 'U20'), ('integer', int)])), axis = 0) for s in All_N_for_i])
        ObjFct_Values_for_N = np.array(sorted(ObjFct_Values_for_N, key= lambda x: x[-1][1]))
        Current_Solution = ObjFct_Values_for_N[0]
        X0 = Current_Solution[:-1]
        move = neighbour_move(Current_Solution, moves_used)
        TL_moves = list(map(lambda d: d[0],Tabu_List))
        while move in TL_moves and len(ObjFct_Values_for_N) > 0 :
            if np.array(sorted(np.array(Save_Solutions_Here, dtype=[('string', 'U20'), ('integer', int)]), key= lambda x: x[0][-1]))[0][-1][1] > Current_Solution[-1][1]: ####### Aspiration criteria
                Tabu_List.remove(Tabu_List[TL_moves.index(move)])
                Tabu_List.append((move, Current_Solution[-1][1]))
                X0 = Current_Solution[:-1]
                Iterations += 1
                break
            ObjFct_Values_for_N = ObjFct_Values_for_N[1:]
            Current_Solution = ObjFct_Values_for_N[0]
            move = neighbour_move(Current_Solution, moves_used)
        else:
            if len(ObjFct_Values_for_N) == 0:
                if (Iteration + 1)%5 == 0:
                    new_len = np.random.randint(10,30)
                    if new_len < Length_of_Tabu_List:
                        Tabu_List = Tabu_List[:new_len]
                    Length_of_Tabu_List = new_len
                Iterations += 1
                continue
            else:
                Tabu_List.append((move, Current_Solution[-1]))
                X0 = Current_Solution[:-1]
                Iterations += 1
        
        if len(Tabu_List) > Length_of_Tabu_List:
            Tabu_List = Tabu_List[1:]
        
        Save_Solutions_Here.append(Current_Solution)

        if (Iteration + 1)%5 == 0:
            new_len = np.random.randint(10,30)
            if new_len < Length_of_Tabu_List:
                Tabu_List = Tabu_List[:new_len]
            Length_of_Tabu_List = new_len

    Save_Solutions_Here = np.array(sorted(Save_Solutions_Here, key= lambda x: x[-1][1]))
    return Save_Solutions_Here

def select_best_feaseble_solution(solutions, shift_stats, ts_horizon, overlap, shift_lenghts, timeframe, demand, max_cap, shift_costs):
    demand_ = pd.DataFrame.from_dict(demand.items()).rename(columns = {0: "Date_time", 1 : "Staffing_level"})
    demand_.set_index("Date_time", inplace = True)
    mapping = {"Monday" : "1", "Tuesday" : "2", "Wednesday" : "3", "Thursday" : "4", "Friday" : "5", "Saturday" : "6", "Sunday" : "7"}
    demand_.index = list(map(lambda x : pd.to_datetime("2023-01-0" + mapping[x.split(" ")[0]] + " " + x.split(" ")[1]), demand_.index.tolist()))
    demand_ = demand_.sort_index().resample("15min").ffill()
    for n, solution in enumerate(solutions):
        solution = [shift for shift in solution[:-1] if shift[1] > 0]
        results, n_assigned = tabularize_results_heu(solution, shift_stats, ts_horizon, overlap, shift_lenghts, timeframe)
        ts_ = list(forecasts.create_horizon_dates("2023-01-01", 1, 15))
        n_assigned = results[["start_num", "end_num", "staff"]].astype(int).copy()
        n_assigned["coverage"] = n_assigned[["start_num", "end_num"]].apply(lambda df: ts_[df["start_num"] : df["end_num"]], axis = 1)
        n_assigned =  n_assigned.explode("coverage")
        n_assigned = n_assigned.groupby("coverage").agg({"staff":"sum"}).sort_index()
        n_assigned.rename({"staff": "Assigned"}, axis = 1, inplace = True)
        n_assigned = n_assigned.sort_index().resample("15min").ffill()
        n_assigned = pd.merge(n_assigned, demand_, right_index = True, left_index = True, how = "outer").ffill()
        n_assigned.reset_index(drop = False, names = ["date"], inplace = True)
        if n_assigned.Assigned.max() <= max_cap:
            return n_assigned, get_final_objs(solution, shift_costs, demand, shift_stats, ts_horizon)[0]
    n = 0
    solution = solutions[n]
    solution = [shift for shift in solution[:-1] if shift[1] > 0]
    results, n_assigned = tabularize_results_heu(solution, shift_stats, ts_horizon, overlap, shift_lenghts, timeframe)
    ts_ = list(forecasts.create_horizon_dates("2023-01-01", 1, 15))
    n_assigned = results[["start_num", "end_num", "staff"]].astype(int).copy()
    n_assigned["coverage"] = n_assigned[["start_num", "end_num"]].apply(lambda df: ts_[df["start_num"] : df["end_num"]], axis = 1)
    n_assigned =  n_assigned.explode("coverage")
    n_assigned = n_assigned.groupby("coverage").agg({"staff":"sum"}).sort_index()
    n_assigned.rename({"staff": "Assigned"}, axis = 1, inplace = True)
    n_assigned = n_assigned.sort_index().resample("15min").ffill()
    n_assigned = pd.merge(n_assigned, demand_, right_index = True, left_index = True, how = "outer").ffill()
    n_assigned.reset_index(drop = False, names = ["date"], inplace = True)
    return n_assigned, get_final_objs(solution, shift_costs, demand, shift_stats, ts_horizon)[0]

def run_heuristic(runs,earliest, latest, shift_lenghts, minimum_night, minimum_day,demand_table, timeframe, demand_weigth, overlap, max_cap, teamsize):
    shift_lenghts = list(map(lambda x: x*(60//timeframe), shift_lenghts))
    ts_horizon, S,\
    demand_dict, shift_costs,\
        t_s_cov, starting_shfits,\
            ending_shifts, possible_shifts,\
                shift_stats = optimization_preprocess(n_weeks = 1,
                                                    earliest_shift = earliest,
                                                    latest_shift = latest,
                                                    allowed_lens = shift_lenghts,
                                                    minimum_night= minimum_night,
                                                    minimum_day = minimum_day,
                                                    demand = demand_table.copy(),
                                                    f = timeframe)


    shift_stats["Start_end"] = shift_stats["Start_end"].apply(lambda x : create_range(x, copy(ts_horizon)))
    meta_data = shift_stats.copy()
    meta_dict = meta_data["Start_end"].to_dict()
    retry = True
    while retry :
        try:
            Initial_Solution = build_initial_solution(earliest = copy(earliest),
                                                            shift_day_match_start = copy(starting_shfits),
                                                            t_time = copy(ts_horizon),
                                                            shift_stats = copy(shift_stats),
                                                            demand = demand_table.copy(),
                                                            shift_day_match_end = copy(ending_shifts),
                                                            teamsize = copy(teamsize),
                                                            meta_dict = copy(meta_dict),
                                                            S = copy(S))
            retry = False
        except:
            pass

    ShiftCosts_df = pd.DataFrame(dict(shift_costs),index=[0])
    
    solutions = TabuSearch(X0 = Initial_Solution, Runs = runs, ShiftCosts_df = ShiftCosts_df, demand = demand_dict.copy(), demand_weight = demand_weigth, S = S, t_time = copy(ts_horizon), shift_stats = shift_stats, meta_dict = meta_dict)
    return select_best_feaseble_solution(solutions, shift_stats, copy(ts_horizon), overlap, shift_lenghts, timeframe, demand_dict, max_cap, shift_costs)

def create_instances(instances, timeframe, horizon, params, earliest, latest, shift_lenghts, minimum_night, minimum_day, teamsize, max_cap, overlap, demand_weigth, runs):
    all_data = []
    for file in os.listdir("..\data\List_Raw\\112_Soer_Oest"):
        file = "..\\data\\List_Raw\\112_Soer_Oest\\" + file
        all_data.append(data_page.import_file(file))

    # len()
    call_types = ["112","02800"]
    ### Prepare
    a, all_data, b, c = prep.prepare_data(all_data, timeframe)
    KPIs = []
    for i in range(instances):
        print("INSTANCE : ", i+1)
        for n in range(len(all_data)):
            data = all_data[n].iloc[-instances*horizon*7*24:]
            call_type = call_types[n]
            predictions = pd.DataFrame(data.iloc[i * len(data)//(instances) : (i+1) * len(data)//(instances)].Calls).rename(columns = {"Calls": "Predictions"})
            service_level = params["service_level"][call_type]
            asa = params["asa"][call_type]
            aht = params["aht"][call_type]
            shrinkage = params["shrinkage"][call_type]
            max_occupancy = params["max_occupancy"][call_type]
            aht  = [int(aht//60)]
            asa  = [asa/60]
            shrinkage  = [shrinkage/100]
            service_level  = [service_level/100]
            max_occupancy  = [max_occupancy/100]
            sub = q_page.prepare_CyclicWeek(predictions, Type = call_type)
            staffing_levels = sim.compute_staffing_levels(  demand = sub,
                                                            aht = aht,
                                                            planning_period = [timeframe],
                                                            asa = asa,
                                                            shrinkage = shrinkage,
                                                            service_level = service_level,
                                                            max_occupancy = max_occupancy,
                                                            Type = call_type)
            if n == 0:
                demand_112 = pd.DataFrame(staffing_levels["Staffing Level"]).rename({"Staffing Level" : "Staffing_level"}, axis = 1)
            else:
                demand_02  = pd.DataFrame(staffing_levels["Staffing Level"]).rename({"Staffing Level" : "Staffing_level"}, axis = 1)
        demand = demand_112 + demand_02
        demand["112"] = demand_112["Staffing_level"]
        demand["02800"] = demand_02["Staffing_level"]
        
        Demand = demand.copy()
        demand_soft = demand.copy()

        # print("Mean Staffing Level: ", demand.Staffing_level.mean())
        # print("Standard Deviation Staffing Level: " , demand.Staffing_level.std())
        # print("Max Staffing Level: " , demand.Staffing_level.min())
        # print("Min Staffing Level: " , demand.Staffing_level.max())
        soft_n_assigned, soft_cost = run_soft(earliest = earliest,
                                                latest = latest,
                                                shift_lenghts = shift_lenghts,
                                                minimum_night = minimum_night,
                                                minimum_day = minimum_day,
                                                timeframe = timeframe,
                                                demand = demand_soft,
                                                teamsize = teamsize,
                                                max_cap = max_cap,
                                                demand_weigth = demand_weigth,
                                                overlap = overlap)

        soft_n_assigned["OverStaffing"] = (soft_n_assigned["Assigned"] - soft_n_assigned["Staffing_level"]).apply(lambda x : max(x, 0))
        soft_n_assigned["UnderStaffing"] = (soft_n_assigned["Staffing_level"] - soft_n_assigned["Assigned"]).apply(lambda x : max(x, 0))
        soft_over = soft_n_assigned["OverStaffing"].mean()
        soft_under = soft_n_assigned["UnderStaffing"].mean()
        print("Model Completed")
        heu_n_assigned, heu_cost = run_heuristic(runs = runs,
                                        earliest = earliest,
                                        latest =  latest,
                                        shift_lenghts =  shift_lenghts,
                                        minimum_night =  minimum_night,
                                        minimum_day =  minimum_day,
                                        demand_table = Demand.copy(),
                                        timeframe =  timeframe,
                                        demand_weigth =  demand_weigth,
                                        overlap =  overlap,
                                        max_cap =  max_cap,
                                        teamsize =  teamsize)
        
        heu_n_assigned["OverStaffing"] = (heu_n_assigned["Assigned"] - heu_n_assigned["Staffing_level"]).apply(lambda x : max(x, 0))
        heu_n_assigned["UnderStaffing"] = (heu_n_assigned["Staffing_level"] - heu_n_assigned["Assigned"]).apply(lambda x : max(x, 0))
        heu_over = heu_n_assigned["OverStaffing"].mean()
        heu_under = heu_n_assigned["UnderStaffing"].mean()
        print("Heuristic Completed")
        
    #     # statistics and/or kpis
        KPIs.append((i + 1 , "Soft", soft_over, soft_under, soft_cost))
        KPIs.append((i + 1 , "Heu", heu_over, heu_under, heu_cost))
    return KPIs


In [27]:
timeframe = 30
horizon = 9
# DEFAULT SCENARIO PARAMETERS - OK 
parameters = {"service_level" : {"112" : 90, "02800" : 80},
        "shrinkage" :     {"112" : 30, "02800" : 30},
        "max_occupancy" : {"112" : 80, "02800" : 80},
        "asa" :           {"112" : 20, "02800" : 180},
        "aht":            {"112" : 320,"02800" : 380}}

earliest = 7
latest = 23
shift_lenghts = [5,6,7,8,9,10]
minimum_night = 3
minimum_day = 4
teamsize = 4
max_cap = 15

instances = 15
overlap = 45
demand_weigth = 0.85
Runs = 60

test = create_instances(instances = instances,
                        timeframe = timeframe,
                        horizon = horizon, 
                        params= parameters,
                        earliest = earliest,
                        latest = latest,
                        shift_lenghts = shift_lenghts,
                        minimum_night = minimum_night,
                        minimum_day = minimum_day,
                        teamsize = teamsize,
                        max_cap = max_cap,
                        overlap = overlap,
                        demand_weigth = demand_weigth,
                        runs = Runs)



two types
INSTANCE :  1
Model Completed
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True
Maximum shift length is respected: True


In [28]:
test#.to_clipboard()

[(1, 'Soft', 0.38095238095238093, 0.8601190476190477, 512434.0),
 (1, 'Heu', 1.667185069984448, 0.5660964230171073, 584416.0),
 (2, 'Soft', 0.34226190476190477, 0.6369047619047619, 484776.0),
 (2, 'Heu', 3.2529761904761907, 0.4523809523809524, 684516.0),
 (3, 'Soft', 0.39880952380952384, 0.5476190476190477, 503374.0),
 (3, 'Heu', 1.3214285714285714, 0.875, 471734.0),
 (4, 'Soft', 0.34523809523809523, 0.7410714285714286, 485614.0),
 (4, 'Heu', 1.2948517940717628, 1.0265210608424338, 508524.0),
 (5, 'Soft', 0.4107142857142857, 0.5982142857142857, 489600.0),
 (5, 'Heu', 2.0386904761904763, 0.2261904761904762, 584096.0),
 (6, 'Soft', 0.36607142857142855, 0.7797619047619048, 485212.0),
 (6, 'Heu', 1.8095238095238095, 0.31845238095238093, 598236.0),
 (7, 'Soft', 0.5476190476190477, 0.6577380952380952, 558914.0),
 (7, 'Heu', 3.7202380952380953, 0.1875, 807096.0),
 (8, 'Soft', 0.22321428571428573, 0.8452380952380952, 493078.0),
 (8, 'Heu', 1.8630952380952381, 0.1875, 621496.0),
 (9, 'Soft', 0.

In [40]:
test = pd.DataFrame.from_records(test, columns= ["Instance", "Method", "Average Overstaffing", "Average Understaffing", "Schedule Cost"])#.to_excel("whatever.xlsx")

In [38]:
instances*horizon/52

2.5961538461538463

In [51]:
test.groupby("Method").mean()

,Instance,Average Overstaffing,Average Understaffing,Schedule Cost
Method,,,,
Heu,8.00,2.07,0.63,"610,844.27"
Soft,8.00,0.37,0.84,"511,856.67"


In [1]:
import pandas as pd 
x = pd.read_clipboard()

In [11]:
# x["Average Overstaffing"] = x["Average Overstaffing"].astype("float")#.str.replace(",", ".")
# x["Average Understaffing"] = x["Average Understaffing"].astype("float")#.str.replace(",", ".")
x["Schedule Cost"] = x["Schedule Cost"].astype("float")#.str.replace(",0", "") #

In [12]:
x

,Instance,District,Method,Average Overstaffing,Average Understaffing,Schedule Cost
0,1,Sør Øst,Soft,0.38,0.86,512434.0
1,1,Sør Øst,Heu,1.67,0.57,584416.0
2,2,Sør Øst,Soft,0.34,0.64,484776.0
3,2,Sør Øst,Heu,3.25,0.45,684516.0
4,3,Sør Øst,Soft,0.40,0.55,503374.0
5,3,Sør Øst,Heu,1.32,0.88,471734.0
6,4,Sør Øst,Soft,0.35,0.74,485614.0
7,4,Sør Øst,Heu,1.29,1.03,508524.0
8,5,Sør Øst,Soft,0.41,0.60,489600.0
9,5,Sør Øst,Heu,2.04,0.23,584096.0


In [18]:
x.groupby(["District", "Method"]).mean()

Instance  Average Overstaffing  Average Understaffing  \
District  Method                                                          
Innlandet Heu         15.5                 2.192                  0.520   
          Soft        15.5                 0.356                  0.959   
Sør Øst   Heu          5.5                 2.023                  0.670   
          Soft         5.5                 0.390                  0.750   

                  Schedule Cost  
District  Method                 
Innlandet Heu          530287.5  
          Soft         436336.0  
Sør Øst   Heu          578200.6  
          Soft         514970.2